# Filtering on CPUs versus GPUs

This tutorial gives a quick introduction to using `gcm-filters` on either CPU's or GPU's.

In [6]:
import gcm_filters
import numpy as np
import xarray as xr

## POP data

We are going to work with data from the CORE-forced Parallel Ocean Program (POP) simulation described in [Johnson et al. (2016)](https://journals.ametsoc.org/view/journals/phoc/46/10/jpo-d-15-0202.1.xml). The corresponding grid is the 0.1 degree nominal resolution POP tripole grid ([Smith et al., 2010](https://www.cesm.ucar.edu/models/cesm1.0/pop2/doc/sci/POPRefManual.pdf)). SST snapshot data and grid variables are stored in the following dataset, which we pull from figshare.

In [8]:
import requests
url = 'https://ndownloader.figshare.com/files/28041441'
file = requests.get(url)
with open('POP_SST.nc', 'wb') as fp:
    fp.write(file.contents)
ds = xr.open_dataset('POP_SST.nc')
ds

AttributeError: 'Response' object has no attribute 'contents'

In this example, we want to **filter** SST **with a fixed factor of 10**, i.e., to nominally 1 degree resolution.

To keep things simple, we will filter with the **simple fixed factor filter**. The `TRIPOLAR_REGULAR_WITH_LAND` Laplacian is suitable for this filter and our data on a tripole grid. This Laplacian only needs a `wet_mask` as a grid variable:

In [ ]:
gcm_filters.required_grid_vars(gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND)

`wet_mask` is a mask that is 1 in ocean T-cells, and 0 in land T-cells. Since we only want to filter temperature in the uppermost level, we only need a 2D `wet_mask`.

In [ ]:
wet_mask = xr.where(ds['KMT']>0, 1, 0)
wet_mask.plot(figsize=(10,6), cbar_kwargs={'label': ''});

We also need the area of the T-cells.

In [ ]:
area = ds.TAREA.where(wet_mask) / 10000  # convert units from cm2 to m2

## Filtering on CPUs

We tell `gcm-filters` to filter on CPU's by providing a `wet_mask` and input data that are **NumPy Arrays** or **Dask Arrays with NumPy chunks** (rather than CuPy Arrays or Dask Arrays with CuPy chunks).

In [ ]:
wet_mask = wet_mask.chunk({'nlat': len(ds.nlat), 'nlon': len(ds.nlon)})  # 1 chunk
wet_mask

In [ ]:
sst = ds.SST.where(wet_mask)
sst = sst.chunk({'nlat': len(ds.nlat), 'nlon': len(ds.nlon)})  # 1 chunk
sst

In [ ]:
area = area.chunk({'nlat': len(ds.nlat), 'nlon': len(ds.nlon)})  # 1 chunk
area

These are our filter specs (see also the section on simple fixed factor filters in [this tutorial](https://gcm-filters.readthedocs.io/en/latest/tutorial_filter_types.html)):

In [ ]:
specs = {
    'filter_scale': 10,
    'dx_min': 1
    'filter_shape': gcm_filters.FilterShape.GAUSSIAN
    'grid_type': gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND
}

We now create our CPU-compatible filter with our **NumPy**-based `wet_mask`.

In [ ]:
filter_cpu = gcm_filters.Filter(grid_vars={'wet_mask': wet_mask}, **specs)
filter_cpu

Next, we filter the **NumPy**-based `SST * area` lazily on the CPU's with the simple fixed factor filter.

In [ ]:
filtered_cpu = filter_cpu.apply(sst * area, dims=['nlat', 'nlon'])
filtered_cpu = filtered_cpu / area
filtered_cpu

Nothing has actually been computed yet. Let's trigger computation.

In [ ]:
%time filtered_cpu = filtered_cpu.compute()

Here is a comparison of unfiltered vs. filtered SST, zoomed into the Gulf Stream region.

In [ ]:
import matplotlib.pyplot as plt

vmin = 5
vmax = 25
yslice = slice(1500, 1750)
xslice = slice(400, 600)

fig,axs = plt.subplots(1,2,figsize=(25,7))
sst.isel(nlat=yslice, nlon=xslice).plot(
    ax=axs[0], 
    vmin=vmin, vmax=vmax, 
    cbar_kwargs={'label': 'degC'}
)
axs[0].set_title('SST', fontsize=18)
filtered_cpu.isel(nlat=yslice, nlon=xslice).plot(
    ax=axs[1], 
    vmin=vmin, vmax=vmax, 
    cbar_kwargs={'label': 'degC'}
)
axs[1].set_title('filtered SST (on CPU)', fontsize=18);

## Filtering on GPUs

We tell `gcm-filters` to filter on GPU's by providing a `wet_mask` and input data that are **CuPy Arrays** or **Dask Arrays with CuPy chunks**. We therefore have to map the NumPy chunks to CuPy chunks.

In [ ]:
import cupy as cp

In [ ]:
wet_mask_gpu = wet_mask.copy()
wet_mask_gpu.data = wet_mask_gpu.data.map_blocks(cp.asarray)
wet_mask_gpu

In [ ]:
sst_gpu = sst.copy()
sst_gpu.data = sst_gpu.data.map_blocks(cp.asarray)
sst_gpu

In [ ]:
tarea_gpu = tarea.copy()
tarea_gpu.data = tarea_gpu.data.map_blocks(cp.asarray)
tarea_gpu

We create the filter with the same filter specs as above, but now with a **CuPy**-based `wet_mask`.

In [ ]:
filter_gpu = gcm_filters.Filter(
    filter_scale=filter_scale,
    dx_min=dx_min,
    filter_shape=filter_shape,
    grid_type=grid_type,
    grid_vars={'wet_mask': wet_mask_gpu}
)
filter_gpu

Filtering of the data works the same way as before, except that we apply our filter to our **CuPy**-based `SST * area`.

In [ ]:
filtered_gpu = filter_gpu.apply(sst_gpu * tarea_gpu, dims=['nlat', 'nlon'])
filtered_gpu = filtered_gpu / tarea_gpu
filtered_gpu

In the next cell, we map the CuPy blocks back to NumPy blocks.

In [ ]:
filtered_gpu.data = filtered_gpu.data.map_blocks(cp.asnumpy)
filtered_gpu

Filtering on GPU is quite a bit faster than on CPU above.

In [ ]:
%time filtered_gpu = filtered_gpu.compute()

Plotting filtered SST gives the same plot as above (that's good!).

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(25,7))
sst.isel(nlat=yslice, nlon=xslice).plot(
    ax=axs[0], 
    vmin=vmin, vmax=vmax, 
    cbar_kwargs={'label': 'degC'}
)
axs[0].set_title('SST', fontsize=18)
filtered_gpu.isel(nlat=yslice, nlon=xslice).plot(
    ax=axs[1], 
    vmin=vmin, vmax=vmax, 
    cbar_kwargs={'label': 'degC'}
)
axs[1].set_title('filtered SST (on GPU)', fontsize=18);

Finally, we convince ourselves that the differences in the CPU- vs. GPU-filtered fields are as small as machine precision.

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(25,7))
(filtered_cpu-filtered_gpu).plot(
    ax=axs[0], 
    cbar_kwargs={'label': 'degC'}
)
axs[0].set_title('Difference: CPU-filtered - GPU-filtered', fontsize=18)
(filtered_cpu-filtered_gpu).isel(nlat=yslice, nlon=xslice).plot(
    ax=axs[1], 
    cbar_kwargs={'label': 'degC'}
)
axs[1].set_title('Difference in Gulf Stream region', fontsize=18);